#Step 1-6

In [ ]:
# Step 1: Import necessary libraries
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/content/abalone.data.csv')
df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [ ]:
# Step 3: Preprocess the data
# Perform one-hot encoding for the 'Gender' column
df = pd.get_dummies(df, columns=['Sex'])
df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,False,False,True
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,False,False,True
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,True,False,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,False,False,True
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,False,True,False


In [ ]:
# Convert boolean values to float
df['Sex_F'] = df['Sex_F'].astype(float)
df['Sex_I'] = df['Sex_I'].astype(float)
df['Sex_M'] = df['Sex_M'].astype(float)
# Convert all columns to numeric type
df = df.apply(pd.to_numeric, errors='coerce')

# Drop rows with missing values
df.dropna(inplace=True)
df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0.0,0.0,1.0
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0.0,0.0,1.0
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1.0,0.0,0.0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0.0,0.0,1.0
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0.0,1.0,0.0


In [ ]:
# Standardize continuous features
continuous_features = ['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight']
scaler = StandardScaler()
df[continuous_features] = scaler.fit_transform(df[continuous_features])

In [ ]:
# Step 4: Split the data into training and testing sets
X = df.drop('Rings', axis=1).values
y = df['Rings'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# # Convert to PyTorch tensors
# X = torch.tensor(X, dtype=torch.float32)
# y = torch.tensor(y, dtype=torch.float32)

# Step 5: Convert datasets into PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# Step 6: Define the neural network class
class AbaloneModel(nn.Module):
    def __init__(self):
        super(AbaloneModel, self).__init__()
        self.fc1 = nn.Linear(10, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = AbaloneModel()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
# Step 8: Train the model
def train_model(model, criterion, optimizer, X_train, y_train, epochs=30):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train.view(-1, 1))
        loss.backward()
        optimizer.step()
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

In [ ]:
train_model(model, criterion, optimizer, X_train, y_train)

Epoch [10/30], Loss: 19.35650634765625
Epoch [20/30], Loss: 10.388575553894043
Epoch [30/30], Loss: 10.285181999206543


In [ ]:
# Step 9: Evaluation function
def evaluate(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        mse = criterion(predictions, y_test.view(-1, 1)).item()
    return mse

# Step 10: Evaluate the model
mse = evaluate(model, X_test, y_test)
print(f'Mean Squared Error on test set: {mse}')

Mean Squared Error on test set: 10.825898170471191


#Step 7

In [ ]:
# Define the neural network class
class AbaloneModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(AbaloneModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the function to train and evaluate the model
def train_evaluate_model(X_train, y_train, X_test, y_test, model, criterion, optimizer, epochs, batch_size):
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        mse = criterion(predictions, y_test.view(-1, 1)).item()
    return mse

# Define hyperparameters to tune
learning_rates = [0.001, 0.01, 0.1]
batch_sizes = [32, 64, 128]
hidden_sizes1 = [32, 64, 128]
hidden_sizes2 = [16, 32, 64]
epochs = 30

# Grid search over hyperparameters
results = []
for lr in learning_rates:
    for batch_size in batch_sizes:
        for hidden_size1 in hidden_sizes1:
            for hidden_size2 in hidden_sizes2:
                model = AbaloneModel(input_size=X
                                     .shape[1], hidden_size1=hidden_size1, hidden_size2=hidden_size2)
                criterion = nn.MSELoss()
                optimizer = optim.SGD(model.parameters(), lr=lr)
                mse = train_evaluate_model(X_train, y_train, X_test, y_test, model, criterion, optimizer, epochs, batch_size)
                results.append({'Learning Rate': lr, 'Batch Size': batch_size, 'Hidden Size 1': hidden_size1, 'Hidden Size 2': hidden_size2, 'MSE': mse})

# Tabulate the results
results_df = pd.DataFrame(results)
print(results_df)
# Sort results by MSE
results_df_sorted = results_df.sort_values(by='MSE')

# Print the combination with the lowest MSE
print("Combination with the lowest MSE:")
print(results_df_sorted.iloc[0])

    Learning Rate  Batch Size  Hidden Size 1  Hidden Size 2       MSE
0           0.001          32             32             16  4.626805
1           0.001          32             32             32  4.632166
2           0.001          32             32             64  4.763348
3           0.001          32             64             16  4.678527
4           0.001          32             64             32  4.689504
..            ...         ...            ...            ...       ...
76          0.100         128             64             32       NaN
77          0.100         128             64             64       NaN
78          0.100         128            128             16  5.214353
79          0.100         128            128             32       NaN
80          0.100         128            128             64       NaN

[81 rows x 5 columns]
Combination with the lowest MSE:
Learning Rate      0.010000
Batch Size        32.000000
Hidden Size 1    128.000000
Hidden Size 2     64

#Step 8

In [ ]:
# Define the neural network class with additional hidden layers
class AbaloneModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(AbaloneModel, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_sizes[0])])
        for i in range(len(hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
        self.output_layer = nn.Linear(hidden_sizes[-1], 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        for hidden_layer in self.hidden_layers:
            x = self.relu(hidden_layer(x))
        x = self.output_layer(x)
        return x

def train_evaluate_model(X_train, y_train, X_test, y_test, model, criterion, optimizer, epochs, batch_size):
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        mse = criterion(predictions, y_test.view(-1, 1)).item()
    return mse

# Define hyperparameters
learning_rate = 0.01
batch_size = 64
hidden_sizes = [64, 32, 16]  # Adding more layers

# Create model instances for SGD and Adagrad optimizers
model_sgd = AbaloneModel(input_size=X.shape[1], hidden_sizes=hidden_sizes)
model_adagrad = AbaloneModel(input_size=X.shape[1], hidden_sizes=hidden_sizes)

# Define criterion and optimizers
criterion = nn.MSELoss()
optimizer_sgd = optim.SGD(model_sgd.parameters(), lr=learning_rate)
optimizer_adagrad = optim.Adagrad(model_adagrad.parameters(), lr=learning_rate)

# Train and evaluate models with SGD and Adagrad optimizers
mse_sgd = train_evaluate_model(X_train, y_train, X_test, y_test, model_sgd, criterion, optimizer_sgd, epochs=100, batch_size=batch_size)
mse_adagrad = train_evaluate_model(X_train, y_train, X_test, y_test, model_adagrad, criterion, optimizer_adagrad, epochs=100, batch_size=batch_size)

# Print MSE for both optimizers
print("MSE with SGD optimizer:", mse_sgd)
print("MSE with Adagrad optimizer:", mse_adagrad)

MSE with SGD optimizer: 4.674018383026123
MSE with Adagrad optimizer: 4.560866832733154


#Step 9

In [ ]:
class AbaloneModel(nn.Module):
    def __init__(self, input_size, hidden_sizes):
        super(AbaloneModel, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_sizes[0])])
        for i in range(len(hidden_sizes) - 1):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
        self.output_layer = nn.Linear(hidden_sizes[-1], 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        for hidden_layer in self.hidden_layers:
            x = self.sigmoid(hidden_layer(x))
        x = self.output_layer(x)
        return x

def train_evaluate_model(X_train, y_train, X_test, y_test, model, criterion, optimizer, epochs, batch_size):
    train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    for epoch in range(epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.view(-1, 1))
            loss.backward()
            optimizer.step()

    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        mse = criterion(predictions, y_test.view(-1, 1)).item()
    return mse

# Define hyperparameters
learning_rate = 0.01
batch_size = 64
hidden_sizes = [64] * 10  # Number of hidden layers = 10, each with 64 nodes

# Create model instances
model_sigmoid = AbaloneModel(input_size=X.shape[1], hidden_sizes=hidden_sizes)

# Define criterion and optimizer
criterion = nn.MSELoss()
optimizer_sigmoid = optim.SGD(model_sigmoid.parameters(), lr=learning_rate)

# Train and evaluate the model with Sigmoid activation
mse_sigmoid = train_evaluate_model(X_train, y_train, X_test, y_test, model_sigmoid, criterion, optimizer_sigmoid, epochs=100, batch_size=batch_size)

# Print MSE for the model with Sigmoid activation
print("MSE with Sigmoid activation:", mse_sigmoid)

MSE with Sigmoid activation: 11.028924942016602
